<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/proportion_vs_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Fragility Metrics Toolkit: Proportion vs Benchmark
# 20-NOV-2025
# Layout:
# Single arm (or agreement vs benchmark when only k, n, p0 are available)
# k    = number of successes
# n    = relevant denominator (n_relevant)
# p0   = benchmark proportion
#
# Input:  {k, n, p0}
# Output: p   (one-sided exact binomial p-value)
#         fr  (BFI/BFQ)
#         nb  (Proportion-NBF)
#
# Notes
# - One-sided exact binomial test:
#       H1: p > p0   (superiority / “above benchmark”)  OR
#       H1: p < p0   (“maximum allowable rate” / “below benchmark”)
# - BFQ is the primary fragility quotient (fr) for this design
# - Proportion-NBF is the primary robustness metric (nb) for single-arm benchmarks
# - α = 0.05 fixed (change ALPHA below if needed)
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit [Software]. Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

# ---------- SciPy dependency ----------

try:
    import scipy
except ImportError:
    try:
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "scipy"])
        import scipy
    except Exception:
        print("Please install scipy: pip install scipy")
        raise

from math import sqrt
from scipy.stats import binomtest

ALPHA = 0.05


# ---------- Core utilities ----------

def binom_one_sided_p(k: int, n: int, p0: float, alternative: str = "greater") -> float:
    """
    One-sided exact binomial p-value:

      alternative = "greater" -> P(X >= k | n, p0), H1: p > p0
      alternative = "less"    -> P(X <= k | n, p0), H1: p < p0
    """
    if alternative not in {"greater", "less"}:
        raise ValueError('alternative must be "greater" or "less".')
    return binomtest(k, n, p0, alternative=alternative).pvalue


def is_significant(p: float, alpha: float = ALPHA) -> bool:
    return p <= alpha


# ---------- BFI / BFQ (Benchmark Fragility) ----------

def compute_bfi_bfq(
    k: int,
    n: int,
    p0: float,
    alpha: float = ALPHA,
    alternative: str = "greater",
):
    """
    Benchmark Fragility Index (BFI) and Benchmark Fragility Quotient (BFQ).

    BFI = minimal number of success/failure toggles (within the single arm)
          that flips the one-sided exact binomial classification vs p0.
    BFQ = BFI / n  (n_relevant = n for single-arm benchmark designs)

    alternative:
      - "greater" : H1: p > p0  (superiority / above benchmark)
      - "less"    : H1: p < p0  (maximum allowable rate / below benchmark)

    Returns: (BFI, BFQ) or (None, None) if no flip is attainable.
    """
    if alternative not in {"greater", "less"}:
        raise ValueError('alternative must be "greater" or "less".')

    p_val = binom_one_sided_p(k, n, p0, alternative=alternative)
    currently_sig = is_significant(p_val, alpha)

    # For each case below we move monotonically along k in the direction
    # that makes the test less or more extreme for the chosen alternative.

    if alternative == "greater":
        if currently_sig:
            # Significant on the right tail: decrease k until it becomes non-significant.
            for d in range(1, k + 1):
                if binom_one_sided_p(k - d, n, p0, "greater") > alpha:
                    return d, d / n
            return None, None
        else:
            # Non-significant: increase k until it becomes significant.
            for d in range(1, n - k + 1):
                if binom_one_sided_p(k + d, n, p0, "greater") <= alpha:
                    return d, d / n
            return None, None

    else:  # alternative == "less"
        if currently_sig:
            # Significant on the left tail: increase k until it becomes non-significant.
            for d in range(1, n - k + 1):
                if binom_one_sided_p(k + d, n, p0, "less") > alpha:
                    return d, d / n
            return None, None
        else:
            # Non-significant: decrease k until it becomes significant.
            for d in range(1, k + 1):
                if binom_one_sided_p(k - d, n, p0, "less") <= alpha:
                    return d, d / n
            return None, None


# ---------- Proportion-NBF (Neutrality Boundary Framework) ----------

def compute_proportion_nbf(k: int, n: int, p0: float):
    """
    Proportion-NBF (robustness metric for single-arm benchmarks).

    Let p̂ = k / n, neutral value p0, and
        S = sqrt( p0 * (1 - p0) / n ).

    Then:
        Proportion-NBF = |p̂ - p0| / (|p̂ - p0| + S)

    This is direction-agnostic (same formula for superiority and max-rate tests).

    Returns nb in [0, 1], or None if undefined (e.g., n <= 0).
    """
    if n <= 0:
        return None

    phat = k / n
    diff = abs(phat - p0)
    S = sqrt(p0 * (1 - p0) / n)

    if diff == 0 and S == 0:
        # Degenerate case where benchmark is perfectly certain and met exactly.
        return 0.0

    return diff / (diff + S)


# ---------- High-level calculator: single proportion vs benchmark ----------

def single_proportion_benchmark(
    k: int,
    n: int,
    p0: float,
    alpha: float = ALPHA,
    alternative: str = "greater",
):
    """
    High-level calculator for single proportion vs benchmark.

    Inputs:
        k   = successes
        n   = denominator (n_relevant)
        p0  = benchmark proportion (0 < p0 < 1)
        alternative:
            "greater" -> H1: p > p0  (superiority / above-benchmark)
            "less"    -> H1: p < p0  (maximum allowable rate / below-benchmark)

    Returns a minimal p–fr–nb structure:
      {
        "p":  <one-sided exact binomial p-value>,
        "fr": {
            "BFI": <Benchmark Fragility Index>,
            "BFQ": <Benchmark Fragility Quotient>
        },
        "nb": {
            "Proportion-NBF": <robustness metric>
        }
      }
    """
    if not (0 <= k <= n):
        raise ValueError("Invalid inputs: require 0 <= k <= n.")
    if not (0 < p0 < 1):
        raise ValueError("Invalid benchmark p0: require 0 < p0 < 1.")
    if alternative not in {"greater", "less"}:
        raise ValueError('alternative must be "greater" or "less".')

    # Baseline p-value for the chosen alternative
    p_val = binom_one_sided_p(k, n, p0, alternative=alternative)

    # Fragility (BFI/BFQ) for the same alternative
    bfi, bfq = compute_bfi_bfq(k, n, p0, alpha=alpha, alternative=alternative)

    # Robustness (Proportion-NBF, direction-agnostic)
    prop_nbf = compute_proportion_nbf(k, n, p0)

    result = {
        "p": p_val,
        "fr": {
            "BFI": bfi,
            "BFQ": bfq
        },
        "nb": {
            "Proportion-NBF": prop_nbf
        }
    }
    return result


# ---------- CLI: minimal p–fr–nb output (matching Program #1 style) ----------

def main():
    print("Proportion vs Benchmark Calculator")
    print("Choose alternative:")
    print("  1 = H1: p > p0  (superiority / above benchmark) [default]")
    print("  2 = H1: p < p0  (maximum allowable rate / below benchmark)\n")

    # ---- FIRST: choose alternative (1 or 2) ----
    alt_choice = input("Select 1 or 2 (press Enter for default = 1): ").strip()
    if alt_choice == "2":
        alternative = "less"      # H1: p < p0
    else:
        alternative = "greater"   # H1: p > p0  (default)

    # ---- THEN: enter k, n, p0 one at a time ----
    k  = int(input("k  (successes): ").strip())
    n  = int(input("n  (denominator): ").strip())
    p0 = float(input("p0 (benchmark, e.g. 0.15): ").strip())

    # Run calculator
    res = single_proportion_benchmark(k, n, p0, alpha=ALPHA, alternative=alternative)

    # Output in p–fr–nb format
    print("\n================ p–fr–nb =================")
    print(f"p  = {res['p']:.6f}")

    fr = res["fr"]
    print("fr:")
    print(f"  BFI = {fr['BFI']}")
    if fr["BFQ"] is not None:
        print(f"  BFQ = {fr['BFQ']:.6f}")
    else:
        print("  BFQ = None")

    nb = res["nb"]
    print("nb:")
    if nb["Proportion-NBF"] is not None:
        print(f"  Proportion-NBF = {nb['Proportion-NBF']:.6f}")
    else:
        print("  Proportion-NBF = None")
    print("==========================================")



if __name__ == "__main__":
    main()
